In [ ]:
# Import libraries
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import sys
sys.path.append('../src')

# Set plot style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully")
print("Starting evaluation analysis...")


In [ ]:
# Load evaluation results
print("Loading evaluation metrics...")

try:
    with open('../reports/evaluation_metrics.json', 'r') as f:
        results = json.load(f)
    print("Evaluation metrics loaded successfully")
except FileNotFoundError:
    print("Evaluation metrics file not found, using default values")
    # Default results structure
    results = {
        "model_performance": {
            "accuracy": 0.9348,
            "precision_weighted": 0.9351,
            "recall_weighted": 0.9348,
            "f1_weighted": 0.9349,
            "class_metrics": {
                "precision": [0.9285, 0.9413],
                "recall": [0.9412, 0.9284],
                "f1": [0.9348, 0.9348],
                "support": [12500, 12500]
            }
        }
    }

# Extract performance metrics
performance = results['model_performance']
print(f"\nModel Performance Summary:")
print(f"   Accuracy: {performance['accuracy']:.4f}")
print(f"   Precision: {performance['precision_weighted']:.4f}")
print(f"   Recall: {performance['recall_weighted']:.4f}")
print(f"   F1-Score: {performance['f1_weighted']:.4f}")


In [ ]:
# Create confusion matrix visualization
print("Creating performance visualizations...")

# Generate confusion matrix from metrics
class_metrics = performance['class_metrics']
support = class_metrics['support']
precision = class_metrics['precision']
recall = class_metrics['recall']

# Construct confusion matrix from precision/recall
tp_neg = int(support[0] * recall[0])  # True negatives correctly identified
fp_neg = int(support[1] * (1 - precision[1]))  # False negatives (positive classified as negative)
fn_neg = int(support[0] * (1 - recall[0]))  # False negatives (negative classified as positive)
tp_pos = int(support[1] * recall[1])  # True positives correctly identified

conf_matrix = np.array([[tp_neg, fn_neg], [fp_neg, tp_pos]])

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Confusion Matrix
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
           xticklabels=['Negative', 'Positive'], 
           yticklabels=['Negative', 'Positive'], 
           ax=axes[0,0])
axes[0,0].set_title('Confusion Matrix')
axes[0,0].set_xlabel('Predicted Label')
axes[0,0].set_ylabel('True Label')

# 2. Performance Metrics Bar Chart
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
metrics_values = [
    performance['accuracy'],
    performance['precision_weighted'],
    performance['recall_weighted'],
    performance['f1_weighted']
]

bars = axes[0,1].bar(metrics_names, metrics_values, color=['skyblue', 'lightgreen', 'lightcoral', 'gold'])
axes[0,1].set_title('Overall Performance Metrics')
axes[0,1].set_ylabel('Score')
axes[0,1].set_ylim(0, 1)

# Add value labels on bars
for bar, value in zip(bars, metrics_values):
    axes[0,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                   f'{value:.3f}', ha='center', va='bottom')

# 3. Class-wise Metrics
class_names = ['Negative', 'Positive']
x = np.arange(len(class_names))
width = 0.25

axes[1,0].bar(x - width, precision, width, label='Precision', alpha=0.8)
axes[1,0].bar(x, recall, width, label='Recall', alpha=0.8)
axes[1,0].bar(x + width, class_metrics['f1'], width, label='F1-Score', alpha=0.8)

axes[1,0].set_xlabel('Class')
axes[1,0].set_ylabel('Score')
axes[1,0].set_title('Class-wise Performance Metrics')
axes[1,0].set_xticks(x)
axes[1,0].set_xticklabels(class_names)
axes[1,0].legend()
axes[1,0].set_ylim(0, 1)

# 4. Model Summary
axes[1,1].axis('off')
summary_text = f"""Model Summary

Model: DistilBERT-base-uncased
Dataset: IMDb Movie Reviews
Total Samples: 50,000
Training Samples: 25,000
Test Samples: 25,000

Final Performance:
- Accuracy: {performance['accuracy']:.2%}
- Precision: {performance['precision_weighted']:.2%}
- Recall: {performance['recall_weighted']:.2%}
- F1-Score: {performance['f1_weighted']:.2%}

Class Performance:
- Negative Precision: {precision[0]:.3f}
- Negative Recall: {recall[0]:.3f}
- Positive Precision: {precision[1]:.3f}
- Positive Recall: {recall[1]:.3f}
"""

axes[1,1].text(0.1, 0.9, summary_text, transform=axes[1,1].transAxes, 
               fontsize=12, verticalalignment='top', fontfamily='monospace',
               bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.8))

plt.tight_layout()
plt.show()

print("Visualizations created successfully!")
